In [0]:
import pandas as pd
import numpy as np
import h5py
from scipy.special import xlogy
from sklearn.metrics import accuracy_score

In [0]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [0]:
def sigmoid(Z):
  
  """Applies sigmoid function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying sigmoid function
  """
  return 1/(1+np.power(np.e, -Z))

In [0]:
def sigmoid_prime(Z):
  
  """Applies differentiation of sigmoid function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying diff of sigmoid function
  """
  return (1-np.power(Z, 2))

In [0]:
def initialize_nn(X):

  """Initializes random weights and bias

  Arguments
  ---------
  X: array-like
    Train Dataset

  Returns
  -------
  dict
    Contains the randomly initialized weights and bias arrays where-
    shape(weights) = (X.Shape[0], 1)
    bias = float value

    The keys for weights and bias arrays in the dict is 'w' and 'b'
  """

  np.random.seed(999)

  w = np.random.randn(X.shape[1], 1) * 0.01
  b = 0

  return {'w': w, 'b': b}

In [0]:
def forward_prop(X, params):

  """Performs forward propagation and calculates output value

    Arguments
    ---------
    X: array_like
      Data
    params: dictionary
      Parameter dict contaning 'w' and 'b'

    Returns
    -------
    dict
      Dictionary contaning 'z' and 'a'
  """

  w = params['w']
  b = params['b']

  z = np.dot(X, w) + b
  a = sigmoid(z)
  
  return {'z': z, 'a': a}

In [0]:
def backward_prop(X, y, cache):

  """Performs forward propagation and calculates output value for a layer

    Arguments
    ---------
    X: array_like
      Data
    y: array_like
      True labels
    cache: dictionary
      Dictionary containing 'z' and 'a'

    Returns
    -------
    dict
      Dictionary containing gradients 'dz', 'dw' and 'db'
  """

  z = cache['z']
  a = cache['a']
  m = X.shape[0]

  dz = a - y
  dw = (1./m)*np.dot(X.T, dz)
  db = (1./m)*np.sum(dz)

  return {'dz': dz, 'dw': dw, 'db': db}

In [0]:
def update_weights(params, changes, learning_rate=0.01):

  """Performs forward propagation and calculates output value for a layer

    Arguments
    ---------
    params: dict
      Dictionary containing 'w' and 'b'
    changes: dict
      Dictionary containing 'dw' and 'db'
    learning_rate: int, float
      Learning rate for the weight update

    Returns
    -------
    dict
      Dictionary containing updated weights and biases

      The keys for weights and bias arrays in the dict is 'w' and 'b'
  """

  w = params['w']
  b = params['b']
  dw = changes['dw']
  db = changes['db']

  w -= learning_rate*dw
  b -= learning_rate*db

  return {'w': w, 'b': b}

In [0]:
def calculate_loss(cache, y):

  """Calculate the entropy loss

    Arguments
    ---------
    cache: dict
      Dictionary contaning 'z' and 'a'
      y: array-like
        True lables

    Returns
    -------
    loss: float
      Entropy loss
  """

  a = cache['a']
  m = y.shape[0]

  return -1/m*np.sum(xlogy(y, a) + xlogy(1-y, 1-a))
  return -1/m*np.sum(y*np.log(a)+(1-y)*np.log(1-a)) 

In [0]:
X_train, y_train, X_test, y_test, classes = load_dataset()

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(209, 64, 64, 3)
(209,)
(50, 64, 64, 3)
(50,)


In [0]:
X_train = X_train.reshape(X_train.shape[0], -1)/255.
X_test = X_test.reshape(X_test.shape[0], -1)/255.
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [14]:
params = initialize_nn(X_train)
for i in range(1000):
  cache = forward_prop(X_train, params)
  loss = calculate_loss(cache, y_train)
  updates = backward_prop(X_train, y_train, cache)
  params = update_weights(params, updates, learning_rate=0.005)

  if i%100 == 0:
    print('Epoch: {}\tLoss:{:.5f}'.format(i, loss), end='')
    train_cache = np.where(cache['a']>0.5, 1, 0)
    print('\tTraining accuracy:{:.5f}'.format(accuracy_score(y_train, train_cache)), end='')
    test_cache = forward_prop(X_test, params)['a']
    test_cache = np.where(test_cache>=0.5, 1, 0)
    print('\tTesting accuracy:{:.5f}'.format(accuracy_score(y_test, test_cache)))

Epoch: 0	Loss:0.72463	Training accuracy:0.45455	Testing accuracy:0.34000
Epoch: 100	Loss:0.58224	Training accuracy:0.68421	Testing accuracy:0.34000
Epoch: 200	Loss:0.46335	Training accuracy:0.81818	Testing accuracy:0.44000
Epoch: 300	Loss:0.37246	Training accuracy:0.89952	Testing accuracy:0.62000
Epoch: 400	Loss:0.32947	Training accuracy:0.91866	Testing accuracy:0.70000
Epoch: 500	Loss:0.30155	Training accuracy:0.92823	Testing accuracy:0.72000
Epoch: 600	Loss:0.27835	Training accuracy:0.93780	Testing accuracy:0.74000
Epoch: 700	Loss:0.25866	Training accuracy:0.94258	Testing accuracy:0.74000
Epoch: 800	Loss:0.24167	Training accuracy:0.95694	Testing accuracy:0.74000
Epoch: 900	Loss:0.22683	Training accuracy:0.96172	Testing accuracy:0.76000
